In [ ]:
sqlContext = SQLContext(sc)

## Load house prices

In [ ]:
text_RDD = sc.textFile("file:///oasis/scratch/comet/zonca/temp_project/hou.txt", minPartitions=48)

In [ ]:
def mapper_parse_lines(line):
    """Parse line into (neighborhoood, price) pair"""
    words = line.split()
    return (words[1], float(words[2]), int(words[0]))

In [ ]:
house_prices_RDD = text_RDD.map(mapper_parse_lines)

In [ ]:
house_prices_RDD.collect()

In [ ]:
house_prices_df = sqlContext.createDataFrame(house_prices_RDD, ["neighborhood", "price", "bedrooms"])

In [ ]:
house_prices_df.show()

In [ ]:
house_prices_df.printSchema()

In [ ]:
from pyspark.ml.feature import StringIndexer

stringIndexer = StringIndexer(inputCol="neighborhood",
                              outputCol="neighborhood_index")


In [ ]:
house_prices_df_indexed = stringIndexer.fit(house_prices_df).transform(house_prices_df)

In [ ]:
house_prices_df_indexed.head()

In [ ]:
from pyspark.sql import Row
from pyspark.mllib.linalg import Vectors

In [ ]:
def create_features(row):
    return Row(label=row.price/1000,
               features=Vectors.dense([row.bedrooms, row.neighborhood_index])
              )
    
house_prices_features = sqlContext.createDataFrame(
    house_prices_df_indexed.map(create_features))

In [ ]:
house_prices_features.show()

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
RandomForestRegressor?

In [ ]:
lr = RandomForestRegressor(numTrees=10, featureSubsetStrategy="all")
# Print out the parameters, documentation, and any default values.
print "LogisticRegression parameters:\n" + lr.explainParams() + "\n"

In [ ]:
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(house_prices_features)

In [ ]:
model1.transform(house_prices_features).take(10)